# 使用使用如excel或csv数据获取时刻表  
<b>处理早先获取到的各种表格样式数据</b>    


## 当前状态  
- 表格基于始发站、终到站、到时、开时和检票口共5个特征来生成，当然有其他的肯定更好  
    - 如交路表等信息较为贫乏,无车站大屏等实时查询能力  
    - 有什么信息生成什么样的时刻表，这里就比较少了    


## 第一部分，定义使用的库函数和常量信息
### 内容
库函数主要分为三大部分：获取数据--处理数据--<del>分析（可视化）数据</del>。即下面引用部分大体分为了自动化和数据处理两部分  
而常量部分主要用于处理数据生成符合游戏格式的时刻表，这部分会随着函数的完善而更为精简  
  
### 待完善/优化的部分
1. 编组映射待交路表信息获取完成后会按照车型映射如长编和重连，即使用下面marshalling字典
2. 速度等级也会按照车型信息重新映射
3. 建立进场路径映射相关常量来完善分场式车站可能的调度问题，提高调度自动化程度
4. .....

In [142]:

import selenium
import pandas  # 当前爬取部分pandas用于承接数据以获取更好的格式
import time
import datetime
import numpy
import random
import traceback

trainType = ["普客", "新空调快速", "新空调特快", "新空调直快", "动车", "高速", "城际"]
stationRegion = ["武汉", "武汉东", "汉口", "武昌"]


Excelpath = "data.xlsx"  # Excel时刻表输入文件路径，如爬取的时刻表或是是复制来的
TextPath = "train.txt"  # 游戏时刻表文件输出路径
# 速度和编组以及类型映射关系,0为普速1为动车2为高速，后续修改


# 车型关系--待完善，后续待交路表部分完成后会使用对应映射而不是上面的简单类型映射
marshalling = {'CHR380BL': [], 'CHR380B': [], 'CHR380B重连': []}


ThisStation = '武汉'
TotalPlat = 20
# track = {}

species = {'新空调普快': ['120', 'LPPPPPP', 0], '新空调快速': ['120', 'LPPPPPP', 0],
           '新空调特快': ['140', 'LPPPPPP', 0], '新空调直快': ['160', 'LPPPPPP', 0],
           '动车': ['200', 'LPPL', 1], '城际': ['200', 'LPPL', 1], '高速': ['300', 'LPPLLPPL', 2]}
species1 = {'K': ['120', 'LPPPPPP', 0], 'T': ['140', 'LPPPPPP', 0], 'Z': ['160', 'LPPPPPP', 0],
            'D': ['200', 'LPPL', 1], 'C': ['200', 'LPPL', 1], 'G': ['300', 'LPPLLPPL', 2]}

station = {'武汉': ['a', -1, 0, 0, 0], '武汉动车所': ['c', 1, 0, 0, 5],
           '京广高速咸宁北方向': ['b', 1, 2, 1, 5], '武九客专葛店南方向': ['d', 1, 2, 1, 4],
           '京广高速孝感北方向': ['e', 1, 1, 2, 10], '沪蓉线红安西方向': ['f', 0, 1, 2, 10],
           '沪蓉线汉口方向': ['h', 0, 1, 2, 5], '丹水池联络线汉口方向': ['i', 0, 1, 2, 7], }

# 线路和车站关系，主要用于从车站-值获取线路-键
routewhchar = {'武汉动车所': ["武汉"],
               '京广高速咸宁北方向': ["咸宁北", "岳阳东", "长沙南"], '武九客专葛店南方向': ["葛店南", "鄂州", "黄冈西", "黄冈东", "黄梅东", "黄石北", "庐山"],
               '京广高速孝感北方向': ["孝感北", "信阳东", "漯河西", "驻马店东", "郑州东"], '沪蓉线红安西方向': ["红安西", "麻城北", "六安", "合肥", "合肥南"],
               '沪蓉线汉口方向': ["汉口"], '丹水池联络线汉口方向': ["汉口"], }
routewh1 = {'c': ["武汉"],
            'b': ["咸宁北", "岳阳东", "长沙南"], 'd': ["葛店南", "鄂州", "黄冈西", "黄冈东", "黄梅东", "黄石北", "庐山"],
            'e': ["孝感北", "信阳东", "漯河西", "驻马店东", "郑州东"], 'f': ["红安西", "麻城北", "六安", "合肥", "合肥南"],
            'f': ["汉口"], 'i': ["汉口"], }

# 试图使用边缘列表来连接线路所和车站
entrancewh = {


}


## 传入车次列表来爬取相应信息

## 第三部分，处理获取到的数据并生成游戏样式的时刻表
这部分最为重要，按照游戏中时刻表格式分为若干部分  
车辆信息部分，如车次编号，速度等级等信息  
停站信息部分，有四部分按顺序分别为 停站编号，停站股道，停站时刻，停站时间。这四部分会单独拆开分别生成  
例： G7 COMMUTER 300 LPPLLPPL X1 : f#1#11:17:00#0 d#6#11:23:00#3 a#1#11:33:00#0  


内容|完成度|实现原理|近期改良计划|远期优化计划
---|---|---|---|---
车辆信息|较为完善|目前为仅按照车次编号和类型生成|近期暂无|完善交路表爬取以修改速度级和编组信息立折车次信息完善
停站信息-停站编号|完善|依照游戏自身设置直接映射|暂无|暂无
停站信息-停站股道|较为完善|看表格信息自己| 无  
停站信息-停站时刻|完善|按照到站时间和出发时间推算|近期暂无|加上线路所映射行车进路部分  
停站信息-停站时间|完善|按照到站时间和出发时间计算|近期暂无|立折车次停站时间等按照交路表推算  
整体上|较为完善|使用apply(lambda) 来进列映射整处理而不是按照行单个处理|暂无  


### 第三部分第〇小部分--处理用函数定义
#### 内容  
目前按照上面输入为上面自动化爬取获取到csv样式的txt文件，输出为按照车站为单位的数据  
1. 首先去除空行和重复值等基本操作进行数据提纯   
2. 在统一处理如得到部分信息如停站时间和始发终到的处理，  
3. 可以获得中心车站的停站时间信息等  
4. 同时预先定义需要使用的处理函数  

#### 实现原理  
1. 整体上以处理dataframe为主，文件→dataframe→文本
2. 大量使用.apply相应处理函数来处理列，尽可能少使用匿名函数来增加可读性
3. excel自身数据相对比较轻量级，处理速度较快且兼容好
4. 停站上就只能通过较为粗暴的映射来实现了，如果存在更好的进路信息完全可以优化  


#### 待完善/优化的部分
近期暂无计划，远期计划加入进路映射以使用线路所来优化自动化  

In [143]:


def dealTime(datafr: pandas.DataFrame) -> pandas.DataFrame:
    # 对于始发车和终到车进行时间处理，默认相同
    for idx, row in datafr.iterrows():
        if row["开时"] == "----":
            row["开时"] = row["到时"]
        elif row["到时"] == "----":
            row["到时"] = row["开时"]

    # 计算停站时间
    datafr["到时"] = pandas.to_datetime(datafr["到时"], errors="ignore")
    datafr["开时"] = pandas.to_datetime(datafr["开时"], errors="ignore")
    # 实发终到设置停站时间
    # pd["停站时间"].replace(0,10,inplace=True)
    return datafr


def stopStTime(stopTime: pandas.Timedelta) -> int:
    a = int(stopTime.seconds/60)  # 处理为数值类型的分钟时间
    # 始发或是终到的时间是随机的
    return a if a > 0 else random.randint(10, 30)


def ModtimeStr(switchTime: pandas.Timestamp) -> str:
    # 处理为仅有时分秒样式的字符串格式的时间
    return switchTime.strftime("%H:%M:%S")


def speMarType(traincode: str) -> str:  # 目前获取编组信息的方式
    til = species1[traincode[0]]  # 处理获取车辆信息字符串部分
    return "{cod} COMMUTER {speed} {mar} X1".format(cod=traincode, speed=til[0], mar=til[1])


def arrLeaTime(t1: pandas.Timestamp, st: list[str], mark: int) -> str:
    # 选取是上一站还是下一站
    tarst = st[mark]
    useTime = station[tarst][4]
    if mark == 0:  # 进场减时间
        res = t1-datetime.timedelta(minutes=useTime)
    else:  # 离场加时间
        res = t1+datetime.timedelta(minutes=useTime)
    # 返回时分秒格式的字符串
    return res.strftime("%H:%M:%S")

# 手动建立映射关系,别的方法不会


def getArrLeaSt(stype, station):
    stcode = ""
    # 多路径车站部分
    if stype == "高速":  # 高速
        ArrLeaveSt = {
            '汉口': '沪蓉线汉口方向',
            '宜昌东': '沪蓉线汉口方向',
            '襄阳东': '沪蓉线汉口方向',
            '成都东': '沪蓉线汉口方向',
            '利川': '沪蓉线汉口方向',
            '重庆北': '沪蓉线汉口方向',
        }
    else:  # 动车城际speedType == "动车" or speedType == "城际"
        ArrLeaveSt = {
            '汉口': '丹水池联络线汉口方向',
            '宜昌东': '丹水池联络线汉口方向',
            '襄阳东': '丹水池联络线汉口方向',
            '成都东': '丹水池联络线汉口方向',
            '利川': '丹水池联络线汉口方向',
            '重庆北': '丹水池联络线汉口方向',
        }

    stcode = ArrLeaveSt.get(station)
    if stcode != None:
        return stcode

    # 常规单路径车站
    ArrLeaveSt = {
        '武汉': '武汉动车所',
        '南宁东': '京广高速咸宁北方向',
        '北京西': '京广高速孝感北方向',
        '南昌西': '武九客专葛店南方向',
        '香港西九龙': '京广高速咸宁北方向',
        '广州南': '京广高速咸宁北方向',

        '深圳北': '武九客专葛店南方向',
        '西安北': '京广高速孝感北方向',
        '温州南': '武九客专葛店南方向',
        '兰州西': '京广高速孝感北方向',
        '长沙南': '京广高速咸宁北方向',
        '郑州东': '京广高速孝感北方向',
        '贵阳北': '京广高速咸宁北方向',
        '昆明南': '京广高速咸宁北方向',
        '上海虹桥': '沪蓉线红安西方向',
        '合肥东': '沪蓉线红安西方向',
        '厦门': '武九客专葛店南方向',
        '南京南': '沪蓉线红安西方向',
        '福州': '武九客专葛店南方向',
        '杭州东': '武九客专葛店南方向',
        '南昌': '武九客专葛店南方向',
        '济南': '京广高速孝感北方向',
        '青岛北': '京广高速孝感北方向',
        '沈阳北': '京广高速孝感北方向',
        '厦门北': '武九客专葛店南方向',
        '大冶北': '武九客专葛店南方向',
        '福州南': '武九客专葛店南方向',
        '黄冈东': '武九客专葛店南方向',
        '安庆': '武九客专葛店南方向',
    }
    stcode = ArrLeaveSt.get(station, "")
    if stcode == "":
        print(f"车站:{station} 未找到")
    return stcode


def checkin(entrance: str) -> int:
    # 终到车等不指定
    # print(entrance,end=" ")
    if type(entrance) == int:
        return 0
    # 检票口A5 非终到车
    try:  # 对于空缺导致的报错
        en = entrance.strip("检票口AB")
    except TypeError:
        return 0

    railrange = [0, 0]
    if en == '1':  # 高速场
        railrange = [1]
    elif en == '2':
        railrange = [2, 3, 4]
    elif en == '3':
        railrange = [5, 6, 7]
    elif en == '4':  # 78道对应京广高速正线
        railrange = [8, 9, 10]
    elif en == '5':
        railrange = [11, 12, 13]
    elif en == '6':
        railrange = [14, 15, 16]  # 存在分界
    elif en == '7':  # 综合场
        railrange = [17, 18, 19]
    else:
        railrange = [20]
    return random.choice(railrange)


def routedispatch(x, mark=0) -> str:
    # 对于分场式车站按线路所分流进场车辆,目前仅设置进场线路所路由
    #print(stoptruck,"\t" ,routearr[mark])
    t = (x["到时"]-datetime.timedelta(minutes=2)).strftime("%H:%M:%S")
    stoptruck = x["检票口"]
    if stoptruck >= 1 and stoptruck <= 14:  # 高速场车辆
        k1 = {"武九客专葛店南方向": f" p#3#{t}#0",  # 武汉南线路所
              "沪蓉线汉口方向": f" n#2#{t}#0",  # 滠口线路所
              "丹水池联络线汉口方向": f" n#2#{t}#0"}
    else:  # 综合场车辆
        k1 = {
            "武九客专葛店南方向": f" p#1#{t}#0",
            "沪蓉线汉口方向": f" n#1#{t}#0",
            "丹水池联络线汉口方向": f" n#1#{t}#0"}

    return k1.get(x["前后站"][mark], "")


def tostrformat(row1):
    res = ""
    try:
        indexst = station.get(row1["车站名称"])
        res = " {0}#{1}#{2}#{3}".format(
            indexst[0], row1["股道"], row1["到达时间"], row1["停站时间"])
    except KeyError:
        print("KeyError in:", str(row1))
    finally:
        return res


### 第三部分第一小部分--爬取信息整体初步处理 
#### 内容
读取表格文件，进行数据清洗等常规操作，修改整理数据格式，由于信息较少不在额外切片直接全部返回 。

#### 实现原理
常规数据清洗方法  

#### 待完善/优化的部分
无  

In [144]:
# 手动建立映射关系,别的方法不会


def processTrains(routefile: str) -> pandas.DataFrame:
    # 处理产生初步所需信息的数据框，车次时刻表部分
    trainRouteFrame = pandas.read_csv(filepath_or_buffer=routefile,
                                      sep=",", encoding="utf8", header=0)
    # print(trainRouteFrame.head())
    trainRouteFrame.drop_duplicates(inplace=True)  # 常规处理
    trainRouteFrame.fillna(value="", inplace=True)
    trainRouteFrame = dealTime(trainRouteFrame)  # 初步处理时间
    trainRouteFrame.reset_index(drop=True, inplace=True)  # 重设索引

    trainRouteFrame["检票口"] = trainRouteFrame["检票口"].apply(lambda x: checkin(x))
    # 处理检票口和映射进场离场路线
    trainRouteFrame["arr"] = trainRouteFrame.apply(
        lambda x: getArrLeaSt(x["类型"], x["始发站"]), axis=1)
    trainRouteFrame["lea"] = trainRouteFrame.apply(
        lambda x: getArrLeaSt(x["类型"], x["终到站"]), axis=1)
    trainRouteFrame["前后站"] = trainRouteFrame.apply(
        lambda x: [x["arr"], x["lea"]], axis=1)  # 合并前后站
    
    # 截取需要的信息
    #trainInfoNeed = trainRouteFrame[["车次", "到时",  "开时", "前后站", "ent"]]
    #
    # trainRouteFrame.rename(columns={"ent": "检票口"}, inplace=True)
    # print(trainRouteFrame)

    return trainRouteFrame


###  第三部分第二小部分 -- 处理生成最终dataframe样式的数据 

#### 内容
按照上面的函数处理数据，最终处理生成有进(离)场时间/股道 等信息的dataframe,之后生成字符串导出    
这里分为两部分，车辆信息的trainframe和路径时刻表的stationframe,  
 'D8176 COMMUTER 200 LPPL X1 : b#2#18:08:00#0 d#0#18:16:00#14 e#0#18:36:00#0 ',

#### 实现原理
> 1. generateTrainInfo 即为文字版时刻表分号前的内容，生成车次信息部分  
>> 1.1. 考虑到内容只需做车次编号，速度等级和编组信息则按照车次首位ktz生成    
>> 2.1. 编组信息尚未准备开始   
> 2. generateArriveLeave 即为文字版时刻表分号后的内容，生成进场和离场信息部分  
>> 2.1. 基于停站的上一站完善进路的刻画
>> 2.2. 本身进场和离场处理基本类似，这里使用一个mark来进行区分
> 3. generateMainSt 即为区域核心车站生成部分  
>> 3.1. 绝大多数信息已经在前面处理完成，这里主要是处理格式问题  
>> 3.2. 车站名称的填充放在最后，防止由于数据全空导致填充失败  

#### 改进计划  
1. 对于分场式车站采用进场线路--对应线路所来映射，加入线路所映射部分


In [145]:

def generateArriveLeave(datafr: pandas.DataFrame, mark) -> pandas.DataFrame:
    # 生成进场/离场信息
    # mark=0为进场，mark=1为离场
    ALStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息
    # 按照信息来映射进场/离场线路
    ALStDF['车站名称'] = datafr["前后站"].apply(lambda x: x[mark])
    # 按照信息来映射进场/离场股道
    try:
        ALStDF['股道'] = ALStDF['车站名称'].apply(lambda x: station[x][2+mark])
    except KeyError as ke:
        print("KeyError?")
    # 时间作差
    
    # 进程默认经过不停车
    ALStDF['停站时间'] = 0
    if mark == 0:  # 对于进场车辆看是否有分场需要
        ALStDF['到达时间'] = datafr.apply(
        lambda x: arrLeaTime(x["到时"], x["前后站"], mark), axis=1)
        ALStDF["线路所"] = datafr.apply(
            lambda x: routedispatch(x, mark), axis=1)
    else:
        ALStDF['到达时间'] = datafr.apply(
        lambda x: arrLeaTime(x["开时"], x["前后站"], mark), axis=1)
    # print(ALStDF.head())
    ALStDF["字符格式"] = ALStDF.apply(lambda x: tostrformat(x), axis=1)
    return ALStDF


def generateMainSt(datafr: pandas.DataFrame, mainst: str):
    # 生成中心停站信息
    MainStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息

    # 计算停站时长，得到整形类型的时长，非浮点数没有.0便于写入文件
    MainStDF["停站时间"] = (datafr["开时"]-datafr["到时"]
                        ).apply(lambda x: stopStTime(x))
    # 计算并规整进场离场和停站时间，将三个到时改为游戏时分秒格式 hh:mm:ss
    MainStDF["到达时间"] = datafr["到时"].apply(lambda x: ModtimeStr(x))

    # 按照相应格式处理车站检票口为对应形式
    # MainStDF["股道"] = datafr["检票口"].apply(lambda x: checkin(x))
    MainStDF["股道"] = datafr["检票口"] #.apply(lambda x: checkin(x))

    # 填充车站为中心车站
    MainStDF["车站名称"].fillna(value=mainst, inplace=True)
    MainStDF["字符格式"] = MainStDF.apply(lambda x: tostrformat(x), axis=1)
    # print(MainStDF)
    return MainStDF


def generateTrainInfo(dataser: pandas.Series) -> pandas.Series:
    # 生成车次信息
    res = dataser.apply(lambda x: speMarType(x))
    # print(res.head())
    return res


## 第四部分 -- 生成对应样式的字符串并写入文件

### 内容  
1. 整合之前的数据来生成符合样式的dataframe数据合集  
2. 生成csv长字符串并按行分割为一个个的端数据  
3. 最终连接处理生成结果，不替换代号便于后续检查    

### 实现原理
整合数据和字符串处理

### 改进计划  
暂无

In [ ]:

def generateStr(fn: str) -> list:
    # 生成游戏样式的字符串
    f = open(file=fn, encoding="utf8", mode="w")
    res = []
    for i in range(0, len(arriveStDF)):
        # format为对应顺序格式
        tf = "{train} :{arrive}{route1}{stop}{leave}".format(
            train=trainDF[i], arrive=arriveStDF["字符格式"][i], route1=arriveStDF["线路所"][i],
            stop=stopStDF["字符格式"][i], leave=leaveStDF["字符格式"][i])
        # print(tf)
        res.append(tf)  # 将结果写入文件
        f.write(tf+"\n")

    f.close()
    return res


at = processTrains("wh1.csv")

# 生成dataframe格式的数据
arriveStDF = generateArriveLeave(at, mark=0)
leaveStDF = generateArriveLeave(at, mark=1)
stopStDF = generateMainSt(at, mainst="武汉")
trainDF = generateTrainInfo(at["车次"])

generateStr("映射版测试.txt")
